# Exam project of group 26
### Group members:
- Baltazar Dydensborg
- Johan Kielgast Ladelund
- Laura Weile
- Simon Juul Hansen

### Research Question:
???

### All of the dependency imports

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import datetime
from tqdm import tqdm
import warnings


## Scraping betting data from Oddsportal.com

In [ ]:
# Example of URLs
newest_season_url = 'https://www.oddsportal.com/soccer/denmark/superliga/results/'
previous_season_url = lambda year_span: f'https://www.oddsportal.com/soccer/denmark/superliga-{year_span}/results/'
previous_season_url('2020-2021')
prev_url_example = 'https://www.oddsportal.com/soccer/denmark/superliga-2020-2021/results/'

In [ ]:
# requesting html data
page = requests.get('https://www.oddsportal.com/soccer/denmark/superliga-2020-2021/results/#/page/2/')
soup = BeautifulSoup(page.content, 'html.parser')
soup.find('body')

## FIRST PROBLEM WITH ODDSPORTAL.COM:
### The HTML data is being retrieved via a JavaScript Query, so we can't directly get it via scraping the website with 'requests'.
### We need to find a solution to getting the HTML data via the JS query, possible via Selenium

# www.football-data.co.uk

In [ ]:
df = pd.read_csv('https://www.football-data.co.uk/mmz4281/2021/E0.csv')

In [ ]:
df

In [ ]:
url_template = lambda year_span: f'https://www.football-data.co.uk/mmz4281/{year_span}/E0.csv'
dict_df = {}

first_years = [f'{x:02}' for x in range(0,21)]
second_years = [f'{x:02}' for x in range(1,22)]
final_years = [x+y for x,y in zip(first_years,second_years)]
final_years

for year in final_years:
    df_temp = pd.read_csv(url_template(year), nrows = 1)
    df_temp2 = pd.read_csv(url_template(year), usecols = df_temp.columns)
    dict_df[year] = df_temp2

In [ ]:
df_temp = pd.read_csv(f'https://www.football-data.co.uk/mmz4281/{final_years[3]}/E0.csv',nrows=1)
df_temp.columns 
df_0203 = pd.read_csv(f'https://www.football-data.co.uk/mmz4281/{final_years[3]}/E0.csv', usecols = df_temp.columns)


In [ ]:
res = requests.get('https://fb.oddsportal.com/ajax-sport-country-tournament-archive/1/4l3rT0aP/X0/1/0/1/')
res.content

# Elpriser fra Nordpool

In [ ]:
url = 'https://www.nordpoolgroup.com/historical-market-data/'
el_res = requests.get(url)
soup = BeautifulSoup(el_res.content, 'html.parser')
soup.find('a')

In [ ]:
df = pd.read_csv('https://www.nordpoolgroup.com/4a2916/globalassets/marketdata-excel-files/elspot-prices_2021_hourly_dkk.xls', sep='\t', skiprows=10)
df

In [ ]:
dls = "https://www.nordpoolgroup.com/4a2916/globalassets/marketdata-excel-files/elspot-prices_2021_hourly_dkk.xls"
resp = requests.get(dls)

with open('2021.xlsx', 'wb') as output:
    output.write(resp.content)
    output.close()

# Vejr data fra DMI API

In [ ]:
# Constants

metObsAPIKey = 'c4503ba1-28d4-45c5-850a-974e98bbb3e0'
climateDataAPIKEy = 'ac27b332-bde2-4138-a53e-f0ca82cf3667'
stationId = 6183

dmi_url= lambda API_KEY: f'https://dmigw.govcloud.dk/v2/metObs/collections/observation/items?limit=300000&stationId=06184&parameterId=temp_dry&bbox=7,54,16,58&datetime=2015-01-01T00:00:00Z/2021-08-01T00:00:00Z&api-key={API_KEY}'
dmi_url_id= lambda API_KEY, id: f'https://dmigw.govcloud.dk/v2/metObs/collections/observation/items?limit=300000&stationId=06184&parameterId={id}&bbox=7,54,16,58&datetime=2015-01-01T00:00:00Z/2021-08-01T00:00:00Z&api-key={API_KEY}'
dmi_url_allID= lambda API_KEY: f'https://dmigw.govcloud.dk/v2/metObs/collections/observation/items?limit=300000&stationId=06184&bbox=7,54,16,58&datetime=2015-01-01T00:00:00Z/2021-08-01T00:00:00Z&api-key={API_KEY}'


In [101]:
def metObsData(stat, start_date, end_date, stationId = '06183', apiKey = metObsAPIKey):
    """
    Function that takes a parameter ID and searches for all observations from DMI
    for the given dates, where the date format is given as 'YYYY-MM-DD'
    """

    # Define constants to be used in the function
    url= lambda API_KEY, id, startDate, endDate, stationId: f'https://dmigw.govcloud.dk/v2/metObs/collections/observation/items?limit=300000&stationId={stationId}&parameterId={id}&bbox=7,54,16,58&datetime={startDate}T00:00:00Z/{endDate}T00:00:00Z&api-key={API_KEY}'
    local_counter = 1
    ErrorCounter = 0
    stationCounter = 0
    stationList = ['06184', '06186', '06187', '06188']

    while True:
        try:
            temp_url = url(apiKey, stat, start_date, end_date, stationId)
            ErrorCounter += 1

            json_dmi = requests.get(temp_url).json()
            prop = [item['properties'] for item in json_dmi['features']]
            temp_df = pd.DataFrame(prop)
            
            temp_df['observed'].drop_duplicates(inplace = True)
            temp_df.index = pd.to_datetime(temp_df['observed'])
            temp_df = temp_df.tz_convert('CET')

            if temp_df.index[-1].date() != datetime.date.fromisoformat(start_date):
                            
                if local_counter == 1:
                    df = temp_df.copy()
                    local_counter += 1
                elif local_counter > 1:
                    df = pd.concat([df,temp_df])
                else:
                    warnings.warn("Something went wrong with the local counter in IF")

                end_date = temp_df.index[-1].date().strftime("%Y-%m-%d")
                continue
            else:
                if local_counter == 1:
                    df = temp_df
                elif local_counter > 1:
                    df = pd.concat([df,temp_df])
                else:
                    warnings.warn("Something went wrong with the local counter in ELSE")

                break
            


        except KeyError as e:

            try:
                stationId = stationList[stationCounter]
                stationCounter += 1
                continue
            except IndexError as i:
                if ErrorCounter == 1:
                    warnings.warn(f"The requested stat {stat} isn't available for any default stations")
                else:
                    warnings.warn(f"The requested stat {stat} isn't available for any default stations for the daterange: {start_date} - {end_date}")
                return None
        
    return df

        
    

In [103]:
def climateData(stat, start_date, end_date, stationId = '06184',apiKey = climateDataAPIKEy):
    """
    Function that takes a parameter ID and searches for all observations from DMI
    for the given dates, where the date format is given as 'YYYY-MM-DD'
    """

    # Define constants to be used in the function
    url= lambda API_KEY, id, startDate, endDate, stationId: f'https://dmigw.govcloud.dk/v2/climateData/collections/stationValue/items?timeResolution=hour&limit=300000&stationId={stationId}&parameterId={id}&datetime={startDate}T00:00:00Z/{endDate}T00:00:00Z&api-key={API_KEY}'
    local_counter = 1
    ErrorCounter = 0
    stationCounter = 0
    stationList = ['06181', '06186', '06187', '06188']

    while True:
        try:
            temp_url = url(apiKey, stat, start_date, end_date, stationId)
            ErrorCounter += 1

            json_dmi = requests.get(temp_url).json()
            prop = [item['properties'] for item in json_dmi['features']]
            temp_df = pd.DataFrame(prop)
            
            temp_df['to'].drop_duplicates(inplace = True)
            temp_df.index = pd.to_datetime(temp_df['to'])
            #temp_df = temp_df.tz_convert('CET')

            if temp_df.index[-1].date() != datetime.date.fromisoformat(start_date):
                            
                if local_counter == 1:
                    df = temp_df.copy()
                    local_counter += 1
                elif local_counter > 1:
                    df = pd.concat([df,temp_df])
                else:
                    warnings.warn("Something went wrong with the local counter in IF")

                end_date = temp_df.index[-1].date().strftime("%Y-%m-%d")
                warnings.warn('Loop loop date wrong')
                continue
            else:
                if local_counter == 1:
                    df = temp_df
                elif local_counter > 1:
                    df = pd.concat([df,temp_df])
                else:
                    warnings.warn("Something went wrong with the local counter in ELSE")

                break
            


        except KeyError as e:

            try:
                stationId = stationList[stationCounter]
                stationCounter += 1
                warnings.warn('Loop loop station wrong')
                continue
            except IndexError as i:
                if ErrorCounter == 1:
                    warnings.warn(f"The requested stat {stat} isn't available for any default stations")
                else:
                    warnings.warn(f"The requested stat {stat} isn't available for any default stations for the daterange: {start_date} - {end_date}")
                return None
        
    return df


In [157]:
def TransformMetObsData(df):
    df = df
    df['hour'] = df.index.hour
    df['date'] = df.index.date
    df = df.drop_duplicates(['parameterId', 'date','hour'])\
            .drop(['created', 'stationId'], axis = 1)\
            .sort_values(by = ['date', 'hour'], ascending = [False, False])\
            .copy()
    
    df_new = df.groupby(['parameterId', 'date', 'hour'])['value'].mean()\
            .unstack(level = 0)\
            .reset_index().rename(columns={df.index.name:None})

    return df_new
    

In [158]:
def transformClimateData(df):
    df = df
    df = df.tz_convert('CET')
    df['hour'] = df.index.hour
    df['date'] = df.index.date
    df = df.drop_duplicates(['parameterId', 'date','hour'])\
            .drop(['calculatedAt', 'created', 'from', 'qcStatus', 'timeResolution', 'validity'], axis = 1)\
            .sort_values(by = ['date', 'hour'], ascending = [False, False])\
            .copy()
    
    df_new = df.groupby(['parameterId', 'date', 'hour'])['value'].mean()\
            .unstack(level = 0)\
            .reset_index().rename(columns={df.index.name:None})

    return df_new

In [102]:
list_of_stats=['wind_min', 'wind_max', 'visibility', 'cloud_cover', 'cloud_height', 'temp_dry', 'wind_speed', 'wind_dir', 'pressure', 'humidity']

dict_df = dict()
for stat in tqdm(list_of_stats):
    dict_df[stat] = metObsData(stat, '2015-01-01', '2021-08-01')

new_df_met = pd.concat(dict_df.values())
transform_df_met = TransformMetObsData(new_df_met)


  0%|          | 0/10 [00:00<?, ?it/s]<ipython-input-101-bfbaca18790e>:65: UserWarning: The requested stat wind_min isn't available for any default stations for the daterange: 2015-01-01 - 2015-01-02
  warnings.warn(f"The requested stat {stat} isn't available for any default stations for the daterange: {start_date} - {end_date}")
100%|██████████| 10/10 [06:39<00:00, 39.91s/it]


In [106]:
list_of_stats_climate =['bright_sunshine', 'mean_radiation', 'mean_pressure', 'mean_wind_dir', 'acc_precip', 'temp_grass', 'mean_relative_hum', 'mean_temp']

dict_df_climate = dict()
for stat in tqdm(list_of_stats_climate):
    dict_df_climate[stat] = climateData(stat, '2015-01-01', '2021-08-01')

new_df_climate = pd.concat(dict_df_climate.values())
transform_df_climate = transformClimateData(new_df_climate)

  0%|          | 0/8 [00:00<?, ?it/s]<ipython-input-103-e4323209ab18>:57: UserWarning: Loop loop station wrong
  warnings.warn('Loop loop station wrong')
 25%|██▌       | 2/8 [00:18<00:54,  9.01s/it]<ipython-input-103-e4323209ab18>:38: UserWarning: Loop loop date wrong
  warnings.warn('Loop loop date wrong')
100%|██████████| 8/8 [01:18<00:00,  9.78s/it]


In [190]:
def metObsPipeline(stats, start_date, end_date):
    dict_df = dict()
    for stat in tqdm(stats):
        dict_df[stat] = metObsData(stat, start_date, end_date)

    new_df_met = pd.concat(dict_df.values())
    transform_df_met = TransformMetObsData(new_df_met)
    return transform_df_met

In [189]:
def climatePipeline(stats, start_date, end_date):
    dict_df = dict()
    for stat in tqdm(stats):
        dict_df[stat] = climateData(stat, start_date, end_date)

    new_df_climate = pd.concat(dict_df.values())
    transform_df_climate = transformClimateData(new_df_climate)
    return transform_df_climate

In [192]:
def merger(df_met, df_climate):
    merge_df = df_met.merge(df_climate, on = ['date', 'hour'], how = 'outer')
    merge_df = merge_df.sort_values(by = ['date', 'hour'], ascending = [False, False]).reset_index(drop = True)
    return merge_df

In [193]:
def total_pipeline(met_stats, climate_stats, start_date, end_date):
    df_met = metObsPipeline(met_stats, start_date, end_date)
    df_climate = climatePipeline(climate_stats, start_date, end_date)
    df_total = merger(df_met, df_climate)
    return df_total